## Inferelator.R ported to python

In [80]:
import csv
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import pandas
from scipy.stats import nbinom
import seaborn
import datetime
import os
import random

In [81]:
proc_time = datetime.datetime.now()

In [82]:
print proc_time.date()

2016-06-13


In [83]:
def intersect(a, b):
    return list(set(a) & set(b))

In [84]:
PARS = {}

In [85]:
PARS['input.dir'] = 'input/bsubtilis'
PARS['exp.mat.file'] = 'expression.tsv'
PARS['tf.names.file'] = 'tf_names.tsv'
PARS['meta.data.file'] = 'meta_data.tsv'
PARS['priors.file'] = None # 'gold_standard.tsv'
PARS['gold.standard.file'] = None
PARS['leave.out.file'] = None
PARS['randomize.expression'] = False
PARS['job.seed'] = 42  # set to None if a random seed should be used
PARS['save.to.dir'] = None
PARS['num.boots'] = 20
PARS['max.preds'] = 10
PARS['mi.bins'] = 10
PARS['cores'] = 8
PARS['delT.max'] = 110
PARS['delT.min'] = 0
PARS['tau'] = 45

# I've modified these parameters to make them lists
PARS['perc.tp'] = [0]
PARS['perm.tp'] = [1]
PARS['perc.fp'] = [0]
PARS['perm.fp'] = [1]
PARS['pr.sel.mode'] = 'random'  # prior selection mode: 'random' or 'tf'
PARS['eval.on.subset'] = False
PARS['method'] = 'BBSR'  # 'BBSR' or 'MEN'
PARS['prior.weight'] = 1
PARS['use.tfa'] = False
PARS['prior.ss'] = False
PARS['output.summary'] = False
PARS['output.report'] = False
PARS['output.tf.plots'] = False
# so['e of the elastic net parameters that are essentially constants;
# on['y override in config script if you know what you are doing
PARS['enet.sparseModels'] = True    # sparser models
PARS['enet.nCv'] = 10               # number of cross-validations
PARS['enet.lambda'] = [0, 1, 100]  # l2 weights
PARS['enet.verbose'] = False        # print progress to screen
PARS['enet.plot.it'] = False        # generate cross-validation plots
PARS['enet.plot.file.name'] = None  # file name for plots
PARS['method'] = 'BBSR'  # which method to use; either 'MEN' or 'BBSR'
PARS['prior.weight'] = 1  # the weight for the priors; has to be larger than 1
# fo[' priors to have an effect
PARS['output.summary'] = True  # write a summary tsv and RData file of network
PARS['output.report'] = True  # create html network report
PARS['output.tf.plots'] = True  # create png files with plots of TFs and targets

In [86]:
# Adding variable to point to top level
PARS['top.level.dir'] = '/Users/ndeveaux/Dev/Inferelator/'

In [87]:
def trivial_meta_data(column_names):
    meta_data = pandas.DataFrame({'condName': column_names})
    meta_data['isTs'] = False
    meta_data['is1stLast'] = 'e'
    meta_data['prevCol'] = None
    meta_data['del.t'] = None
    return meta_data

In [88]:
# I removed the read RData capability from read_input
def read_input(input_dir, exp_mat_file, tf_names_file, meta_data_file, 
                       priors_file, gold_standard_file, leave_out_file, randomize_expression):
  IN = {}
  
  print('Reading input: expression matrix')
  IN['exp.mat'] = pandas.read_csv(os.path.join(input_dir, exp_mat_file), header=0, sep='\t')
  
  print('Reading TF names')
  IN['tf.names'] = list(set(pandas.read_csv(os.path.join(input_dir, tf_names_file), header=None, squeeze=True)))
  
  IN['tf.with.expr'] = intersect(IN['tf.names'], IN['exp.mat'].index)

  print('reading metadata')
  if meta_data_file: 
      IN['meta.data'] = pandas.read_csv(os.path.join(input_dir, meta_data_file), 
                                 header=0, sep='\t')    
  # no meta data given - assume all steady state measurements
  else: 
      IN['meta.data'] = trivial_meta_data(IN['exp.mat'].columns)

  
  print('reading leave-out file')
  # if there is a leave-out file, ignore some conditions
  if leave_out_file:
    leave_out = pandas.read_csv(os.path.join(input_dir, leave_out_file), header=None, squeeze=True)
    print('Leaving out the following conditions: ' + leave_out)
    print '{} of {} are present in the expression data'.format(
        len(intersect(leave_out, list(IN['exp.mat'].columns))), len(leave_out))
    # TODO implement leaveout
  
  
  print('reading priors matrix')
  IN['priors.mat'] = None
  if priors_file:
    IN['priors.mat'] = pandas.read_csv(os.path.join(input_dir, priors_file), header=0)
    
    # TODO write the reshape prior function (currently implement in R_scripts/utils.R)
    # IN$priors.mat <- reshape.prior(IN$priors.mat, rownames(IN$exp.mat), IN$tf.names)
  

  print('reading gold standard matrix')
  IN['gs.mat'] = None
  if gold_standard_file:
    IN['gs.mat'] = pandas.read_csv(os.path.join(input_dir, gold_standard_file),
                                   header=0, sep='\t')
    # TODO write the reshape prior function (currently implement in R_scripts/utils.R)      
    #IN$gs.mat <- reshape.prior(IN$gs.mat, rownames(IN$exp.mat), IN$tf.names)
  
  print(' ... done.')
  
  if randomize_expression:
    print('randomize_expression is set to TRUE; randomizing expression matrix ...')
    old_df = IN['exp.mat']
    new_rows = np.random.permutation(IN['exp.mat'].shape[0])
    new_cols = np.random.permutation(IN['exp.mat'].shape[1])
    for row in range(IN['exp.mat'].shape[0]):
        for col in range(IN['exp.mat'].shape[1]):
            IN['exp.mat'].ix[row, col] = old_df.ix[new_rows[row], new_cols[col]]
    print(' done.\n')
  
  return IN

In [89]:
 # TODO: figure out if this is needed, if not, remove it
    # create dummy clusterStack - a real clusterStack is only needed when inferring 
    # on bi-clusters
   # clusterStack <- trivial.cluster.stack(IN$exp.mat)

In [90]:
IN = read_input(os.path.join(PARS['top.level.dir'], PARS['input.dir']),PARS['exp.mat.file'], PARS['tf.names.file'], PARS['meta.data.file'], 
                       PARS['priors.file'], PARS['gold.standard.file'], PARS['leave.out.file'], PARS['randomize.expression'])

Reading input: expression matrix
Reading TF names
reading metadata
reading leave-out file
reading priors matrix
reading gold standard matrix
 ... done.


In [91]:
if not PARS['save.to.dir']:
  PARS['save.to.dir'] = os.path.join(PARS['top.level.dir'], PARS['input.dir'],str(proc_time.date()))
print "Output dir: " + PARS['save.to.dir']
if not os.path.exists(PARS['save.to.dir']):
  os.makedirs(PARS['save.to.dir'])
elif os.path.exists(os.path.join(PARS['save.to.dir'], "/params_and_input")):
  print 'The output file {} already exists. Exiting.'.format(os.path.join(PARS['save.to.dir'], "/params_and_input"))
  # TODO find a way to Exit here


Output dir: /Users/ndeveaux/Dev/Inferelator/input/bsubtilis/2016-06-13


## Creating design and response matrices

In [92]:
# design matrix is same as exp.mat leaving out last time points
# response matrix is same as design for steady state; linear interpolation else
def design_and_response(meta_data, exp_mat, delT_min, delT_max, tau):
    cond = meta_data['condName']
    prev = meta_data['prevCol']
    delt = meta_data['del.t'] 
    
    # the following operations (setting a value on a copy of a slice from a DataFrame) are not allowed with pandas
    # break time series if del.t is larger than delT.max
    # prev[delt > delT.max] <- NA
    # delt[delt > delT.max] <- NA
    
    if set(meta_data['condName']) != set(exp_mat.columns):
        print 'There are set differences between cond and exp mat'
        
        # Check that the condNames and the exp_mat columns are in sync:
        mismatched_condNames = meta_data['condName'][meta_data['condName'] != exp_mat.columns]
        
        # This actually discovered a crucial difference between R data frames and pandas:
        # R dataframes allow there to be two columns with the same name, but pandas won't 
        # Key example: Mt0_1 exists twice in expression data
        if len(mismatched_condNames) > 0:
            print "There are rows in expression matrix that don't match metadata"
            print mismatched_condNames
            correct_col_names = list(exp_mat.columns[meta_data['condName'] != exp_mat.columns])
            print correct_col_names
            # These can be set using the loc function:
            meta_data['condName'][meta_data['condName'] != exp_mat.columns] = correct_col_names
            
        mismatched_condNames = meta_data['condName'][meta_data['condName'] != exp_mat.columns]
        if len(mismatched_condNames) > 0:
            print 'Rewrite Failed'
    
    if set(meta_data['condName']) != set(exp_mat.columns):
        print 'There are still set differences between cond and exp mat'
    else:
        print 'All good'
    
    # Handle steady state conditions first
    # Steady state is defined as not having a previous state, and not being any other state's previous state
    steady = prev.isnull() & (cond.isin(prev) != True)
    
    # Having run the R code, the only difference I see in the des_matrix from exp_is the ordering. 
    # Let's try that for now
    des_matrix = pandas.DataFrame(exp_mat[cond[steady]]) + pandas.DataFrame(exp_mat[cond[steady != True]])
    res_matrix = pandas.DataFrame(des_matrix)
    
    # This creates a 4218 row by  242 column matrix whose values are columns going from 1... 242
    # resp.idx <- t(matrix(1:ncol(res.mat), ncol(res.mat), nrow(exp.mat)))
    
    # Having run the R code, the only difference I see in the des_matrix from exp_is the ordering. 
    return des_matrix, res_matrix
        


In [93]:
des_matrix, res_matrix = design_and_response(IN['meta.data'], IN['exp.mat'], PARS['delT.min'], 
                               PARS['delT.max'], PARS['tau'])


There are set differences between cond and exp mat
There are rows in expression matrix that don't match metadata
85    Mt0_1
88    Mt0_2
91    Mt0_3
Name: condName, dtype: object
['Mt0_1.1', 'Mt0_2.1', 'Mt0_3.1']
All good


/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


## Parse Priors

In [73]:
def get_priors(exp_mat, tf_names, priors_mat, gs_mat, eval_on_subset, 
                      job_seed, perc_tp, perm_tp, perc_fp, perm_fp, sel_mode):
    if priors_mat is None:
        # Create a zero'd out DataFrame
        priors_mat = pandas.DataFrame([[0 for x in range(len(tf_names))] for y in range(len(exp_mat.index))])
        priors_mat.columns = tf_names
        priors_mat.index = exp_mat.index
        
        
    # priors.pars is list of prior parameters; every entry is a vector of four
    # elements: perc.tp, tp permutation number, perc.fp, fp permutation number
    priors_parameters = []
    
    # I tried to implement the length check but it seems silly in python since these aren't even vectors, they're scalars.
    
    lengths = [len(x) for x in [perc_tp, perm_tp, perc_fp, perm_fp]]
    print lengths
    if len(set(lengths)) != 1:
        print """Error parsing prior parameters: perc.tp, perm.tp, perc.fp, perm.fp \
        don't have the same length."""
        
    for i in range(next(iter(set(lengths)))):
        # Get the modulo remainder
        rmndr = max(perm_tp[i], perm_fp[i]) % min(perm_tp[i], perm_fp[i])
        if (rmndr != 0):
            print"""Error parsing prior parameters: Larger number of permutations is \
            not multiple of smaller number."""
        priors_parameters.append([perc_tp, perm_tp, perc_fp, perm_fp])
        
    for pp in priors_parameters:
        # priors[[i]] <- Matrix(0, nrow(priors.mat), ncol(priors.mat), dimnames=dimnames(priors.mat))
        prior_names = 'frac_tp_{}_perm_{}--frac_fp_{}_perm_{}'.format(pp[0], pp[1], pp[2], pp[3])
        
    # Ignore the hundreds of lines of R code in priors.R
    
    return priors_mat, prior_names
        

In [74]:
priors_mat, prior_name = get_priors(IN['exp.mat'], IN['tf.names'], IN['priors.mat'], IN['gs.mat'], 
                       PARS['eval.on.subset'], PARS['job.seed'], PARS['perc.tp'],
                       PARS['perm.tp'], PARS['perc.fp'], PARS['perm.fp'], 
                       PARS['pr.sel.mode'])

[1, 1, 1, 1]


In [75]:
priors_mat

,BSU31410,BSU29740,BSU27890,BSU40800,BSU37650,BSU05370,BSU21780,BSU05700,BSU39430,BSU02970,...,BSU02370,BSU12560,BSU24020,BSU07220,BSU15320,BSU29400,BSU27170,BSU09520,BSU25490,BSU17850
BSU00370,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
BSU08100,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
BSU01810,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
BSU24250,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
BSU36020,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
BSU23590,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
BSU33970,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
BSU25810,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
BSU05330,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
BSU26720,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [76]:
6 % 5

1

## Bootstrap

In [94]:
# set up the bootstrap permutations
# By sampling with replacement from the num cols of the result matrix
col_range = range(len(res_matrix.columns))
IN['bs.pi'] = pandas.DataFrame([[random.choice(col_range) for x in col_range] for y in range(PARS['num.boots'])])

In [95]:
IN['bs.pi']

,0,1,2,3,4,5,6,7,8,9,...,259,260,261,262,263,264,265,266,267,268
0,169,236,109,1,113,184,247,223,83,218,...,258,84,85,205,264,171,128,161,104,47
1,206,19,204,252,1,96,192,99,154,7,...,232,98,119,195,132,13,191,220,43,134
2,168,163,209,196,255,207,244,158,249,121,...,188,61,123,20,218,87,161,131,239,238
3,129,222,91,162,165,45,132,203,132,24,...,100,16,250,6,79,213,213,160,76,62
4,218,141,152,23,74,66,94,161,39,177,...,176,111,6,138,249,239,97,77,211,247
5,171,19,20,94,29,37,221,60,179,133,...,165,92,197,74,235,150,266,40,243,194
6,209,12,131,196,10,53,90,202,241,47,...,12,114,68,56,172,252,79,213,8,83
7,213,267,239,120,225,107,37,20,55,48,...,176,50,123,74,233,228,67,56,168,41
8,152,77,210,32,38,121,202,94,233,203,...,219,181,245,166,216,212,182,250,14,94
9,94,116,19,183,146,268,51,168,151,155,...,215,175,202,61,143,116,229,268,6,114


## TFA specific initialization
Skipping for now, since use TFA set to false

## Main Loop

In [79]:
IN['grouped.pred'] = []


In [476]:
IN['priors']

KeyError: 'priors'

In [ ]:
for b in bootstrap:
    prior_temp = prior_matrix
    # set a random sample of the edges to zero
    

In [224]:
me = pandas.DataFrame({'condName': IN['exp.mat'].columns})

In [265]:
delt = IN['meta.data']['del.t']

In [269]:
prev = IN['meta.data']['prevCol']

In [266]:
delT_max = PARS['delT.max']

In [290]:
mismatched_condNames = IN['meta.data']['condName'][IN['meta.data']['condName'] != IN['exp.mat'].columns]

In [349]:
mismatched_condNames = meta_data['condName'][meta_data['condName'] != exp_mat.columns]

In [354]:
correct_col_names = list(exp_mat.columns[meta_data['condName'] != exp_mat.columns])

In [437]:
steady = prev.isnull() & (cond.isin(prev) != True)

In [444]:
res_matrix = pandas.DataFrame(exp_mat[cond[steady]])

In [448]:
des_matrix = pandas.DataFrame(exp_mat[cond[steady]]) + pandas.DataFrame(exp_mat[cond[steady != True]])

In [357]:
meta_data['condName'][meta_data['condName'] != exp_mat.columns] = correct_col_names

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [450]:
list(des_matrix.columns)

['B36_1',
 'B36_2',
 'B60_1',
 'B60_2',
 'BC/t_1',
 'BC/t_2',
 'BC_1',
 'BC_2',
 'BI_1',
 'BI_2',
 'BMM_1',
 'BMM_2',
 'BMM_3',
 'BT_1',
 'BT_2',
 'C30_1',
 'C30_2',
 'C90_1',
 'C90_2',
 'Cold_1',
 'Cold_2',
 'Cold_3',
 'Diami_1',
 'Diami_2',
 'Diami_3',
 'Etha_1',
 'Etha_2',
 'Etha_3',
 'Fru_1',
 'Fru_2',
 'Fru_3',
 'G+S_1',
 'G+S_2',
 'G+S_3',
 'G135_1',
 'G135_3',
 'G150_1',
 'G150_2',
 'G150_3',
 'G180_1',
 'G180_2',
 'G180_3',
 'GM+10_2',
 'GM+10_3',
 'GM+120_1',
 'GM+120_3',
 'GM+150_1',
 'GM+150_2',
 'GM+150_3',
 'GM+15_1',
 'GM+15_2',
 'GM+15_3',
 'GM+25_1',
 'GM+25_2',
 'GM+25_3',
 'GM+45_1',
 'GM+45_2',
 'GM+45_3',
 'GM+5_1',
 'GM+5_2',
 'GM+5_3',
 'GM+60_1',
 'GM+60_2',
 'GM+60_3',
 'GM+90_1',
 'GM+90_2',
 'GM+90_3',
 'GM-0.1_1',
 'GM-0.1_2',
 'GM-0.1_3',
 'GM-0.2_1',
 'GM-0.2_2',
 'GM-0.2_3',
 'Glu_1',
 'Glu_2',
 'Glu_3',
 'Glucon_1',
 'Glucon_2',
 'Glucon_3',
 'Gly_1',
 'Gly_2',
 'Gly_3',
 'H2O2_1',
 'H2O2_2',
 'H2O2_3',
 'HPh_1',
 'HPh_2',
 'HPh_3',
 'Heat_1',
 'Heat_2',


In [310]:
IN['exp.mat'].columns[IN['meta.data']['condName'] != IN['exp.mat'].columns]

Index([u'Mt0_1.1', u'Mt0_2.1', u'Mt0_3.1'], dtype='object')

In [319]:
IN['meta.data']['condName'][74:91]


74       SMM_3
75      Salt_3
76       Mt0_1
77     M0t45_1
78     M0t90_1
79       Mt0_2
80     M0t45_2
81     M0t90_2
82       Mt0_3
83     M0t45_3
84     M0t90_3
85       Mt0_1
86    M40t45_1
87    M40t90_1
88       Mt0_2
89    M40t45_2
90    M40t90_2
Name: condName, dtype: object

In [48]:
exp_mat_file = PARS['exp.mat.file']

In [49]:
IN['tf_names'] = pandas.read_csv(os.path.join(input_dir, tf_names_file), header=1)

In [312]:
other = pandas.read_csv(os.path.join(input_dir, exp_mat_file), header=0, sep='\t')

In [55]:
list(set(pandas.read_csv(os.path.join(input_dir, tf_names_file), header=None)[0]))

['BSU31410',
 'BSU29740',
 'BSU27890',
 'BSU40800',
 'BSU37650',
 'BSU05370',
 'BSU21780',
 'BSU05700',
 'BSU39430',
 'BSU02970',
 'BSU13340',
 'BSU26350',
 'BSU03960',
 'BSU40410',
 'BSU05670',
 'BSU04160',
 'BSU07390',
 'BSU05170',
 'BSU25200',
 'BSU12510',
 'BSU13150',
 'BSU07590',
 'BSU03470',
 'BSU40670',
 'BSU08730',
 'BSU40710',
 'BSU38600',
 'BSU03170',
 'BSU03850',
 'BSU38880',
 'BSU33660',
 'BSU35200',
 'BSU15090',
 'BSU30150',
 'BSU27120',
 'BSU39990',
 'BSU09380',
 'BSU05180',
 'BSU34630',
 'BSU24520',
 'BSU37310',
 'BSU24610',
 'BSU01540',
 'BSU37550',
 'BSU22120',
 'BSU37620',
 'BSU33990',
 'BSU01690',
 'BSU03750',
 'BSU40540',
 'BSU40010',
 'BSU36300',
 'BSU08810',
 'BSU26580',
 'BSU29110',
 'BSU18760',
 'BSU10840',
 'BSU40130',
 'BSU00370',
 'BSU22440',
 'BSU17080',
 'BSU32560',
 'BSU11500',
 'BSU05060',
 'BSU05270',
 'BSU05320',
 'BSU09560',
 'BSU38450',
 'BSU08250',
 'BSU34180',
 'BSU25100',
 'BSU26870',
 'BSU12000',
 'BSU36600',
 'BSU13670',
 'BSU03880',
 'BSU05290',

In [58]:
IN['exp.mat'] = pandas.read_csv(os.path.join(input_dir, exp_mat_file), header=0, sep='\t')

In [65]:
IN['exp.mat'][[0]].rows

AttributeError: 'DataFrame' object has no attribute 'rows'

In [72]:
IN['tf.with.expr'] = intersect(IN['tf.names'], IN['exp.mat'].index)

In [71]:
IN['tf.names'] = list(set(pandas.read_csv(os.path.join(input_dir, tf_names_file), header=None)[0]))

In [74]:
len(IN['tf.with.expr'])

239

In [382]:
prev = meta_data['prevCol']